In [2]:
import os
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc # import scanpy to handle our AnnData 

adata = ad.read_h5ad("../data/GSE194122_openproblems_neurips2021_cite_BMMC_processed.h5ad")
gex = adata[:,adata.var["feature_types"] == "GEX"]
adt = adata[:,adata.var["feature_types"] == "ADT"]

/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/envs/multiomics/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adt_md = adt.obs[["cell_type", "batch", "is_train"]].reset_index(drop = True)
adt_md["CT_id"] = adt_md.cell_type.cat.codes
adt_md["split"] = np.where(adt_md["is_train"] == "train", "train", np.where(adt_md["is_train"] == "test", "val", "test"))
adt_df = pd.concat([pd.DataFrame(adt.X.toarray()), adt_md], axis = 1)
adt_df.columns = adt_df.columns.astype("str")

In [9]:
adt_md_test = adt_md.loc[adt_md.split == "test"]

In [12]:
adt_md_test.CT_id.value_counts()

CT_id
15    2129
3      729
5      590
25     520
35     421
17     411
27     375
13     326
16     239
26     204
28     197
18     197
19     187
44     173
40     166
34     165
8      163
22     162
29     152
38     149
4      131
7      123
12     122
11     110
10      94
6       93
0       82
23      78
9       77
24      75
42      69
21      61
1       55
37      54
30      27
31      25
32      18
20      17
43      17
2       10
33       9
41       7
36       6
14       5
Name: count, dtype: int64

In [8]:
adt_md.cell_type.cat.categories[41]

'dnT'

In [12]:
print("scaling")

sc.pp.scale(gex, max_value=10)

print("scaling complete")

sc.tl.pca(gex, n_comps=200, svd_solver="arpack")

gex_md = gex.obs[["cell_type", "batch", "is_train"]].reset_index(drop = True)
gex_md["CT_id"] = gex_md.cell_type.cat.codes
gex_md["split"] = np.where(gex_md["is_train"] == "train", "train", np.where(gex_md["is_train"] == "test", "val", "test"))

gex_df = pd.concat([pd.DataFrame(gex.obsm["X_pca"]), gex_md], axis = 1)

print(gex_df.columns)

gex_df.to_parquet("data/gex_pca_200.parquet")

scaling


/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/envs/multiomics/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:842: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


scaling complete


ValueError: Length mismatch: Expected axis has 205 elements, new values have 139 elements

In [15]:
gex_df.to_parquet("data/gex_pca_200.parquet")